In [1]:
import os
import sys

import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
import scvelo as scv
from scipy import stats
from scipy.sparse import csr_matrix

import celloracle as co

In [2]:
%matplotlib inline

In [3]:
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})

In [4]:
scv.settings.verbosity = 3  # show errors(0), warnings(1), info(2), hints(3)
scv.settings.presenter_view = True  # set max width size for presenter view
scv.set_figure_params('scvelo')  # for beautified visualization

In [5]:
# Load file with GRN info
oracle = co.load_hdf5("./Pancreas_grn_sim.celloracle.oracle")

In [12]:
#imputed count
gem_imputed = pd.DataFrame(oracle.adata.layers['imputed_count'], columns = oracle.adata.var.index.values, index=oracle.adata.obs.index.values)

In [63]:
#start of points (observations)
gem_sim_input = oracle.adata.layers['imputed_count'].copy()

In [14]:
#alpha to perform Ridge Regression
alpha = 10

In [16]:
#Cluster labels per cell
cluster_info = oracle.adata.obs['clusters']

Beta cluster as example

In [17]:
#cells in cluster Beta
cells_in_beta = (cluster_info == 'Beta')

In [18]:
#imputed count of cells in cluster Beta
gem = gem_imputed[cells_in_beta]

In [19]:
#all genes
genes = gem.columns

In [20]:
#list of target genes from cluster with their respective TF's (in cluster Beta)
TFdict=oracle.cluster_specific_TFdict['Beta']

In [24]:
#target genes
all_genes_in_dict = list(set(gem.columns).intersection(list(TFdict.keys())))

In [26]:
#init with zero all 2000 genes
zero_ = pd.Series(np.zeros(len(genes)), index=genes)

In [35]:
from sklearn.linear_model import Ridge

In [36]:
def get_coef(target_gene):
        tmp = zero_.copy()

        # define regGenes
        reggenes = TFdict[target_gene]
        #reggenes = intersect(reggenes, genes)
        reggenes = list(set(reggenes).intersection(list(genes)))

        if target_gene in reggenes:
            reggenes.remove(target_gene)

        if len(reggenes) == 0 :

            tmp[target_gene] = 1
            return(tmp)


        # prepare learning data
        Data = gem[reggenes]
        Label = gem[target_gene]


        # model fitting
        model = Ridge(alpha=alpha, random_state=123)
        model.fit(Data, Label)

        tmp[reggenes] = model.coef_
        intercept_ = model.intercept_ 

        return tmp, intercept_

In [52]:
reggenes = TFdict['Ppp1r1a']
reggenes

array(['Peg3', 'Vdr', 'Nr4a2', 'Ovol2', 'Zfp467', 'Relb', 'E2f1', 'Hnf4a',
       'Ddit3', 'Zfp90', 'Hivep1', 'Tfcp2', 'Nr1h4'], dtype=object)

In [53]:
reggenes = list(set(reggenes).intersection(list(genes)))
reggenes

['Tfcp2',
 'Zfp90',
 'Nr4a2',
 'Zfp467',
 'Vdr',
 'Nr1h4',
 'Relb',
 'Ovol2',
 'Ddit3',
 'E2f1',
 'Hnf4a',
 'Hivep1',
 'Peg3']

In [54]:
Data = gem[reggenes]
Data.head()

,Tfcp2,Zfp90,Nr4a2,Zfp467,Vdr,Nr1h4,Relb,Ovol2,Ddit3,E2f1,Hnf4a,Hivep1,Peg3
AAACGGGCAAAGAATC,0.062066,0.092348,0.119933,0.127558,0.218114,0.008502,0.013756,0.122148,0.100962,0.038661,0.067924,0.061421,1.588050
AAACGGGGTACAGTTC,0.080351,0.059268,0.063167,0.042520,0.113693,0.017316,0.019812,0.119374,0.087693,0.063359,0.117505,0.087700,1.418761
AAACGGGTCGCATGGC,0.030334,0.076545,0.107376,0.120368,0.146049,0.031592,0.000000,0.161057,0.156562,0.028561,0.069866,0.065906,1.422853
AAAGATGAGTCATGCT,0.058535,0.050540,0.051827,0.051987,0.131108,0.019153,0.020692,0.149305,0.139800,0.031537,0.060409,0.088753,1.326516
AAAGATGAGTCTTGCA,0.100374,0.050300,0.059903,0.057831,0.103427,0.011138,0.007248,0.169970,0.180799,0.065875,0.089835,0.078189,1.374747


In [55]:
Label = gem['Ppp1r1a']
Label.head()

AAACGGGCAAAGAATC    0.522952
AAACGGGGTACAGTTC    2.088425
AAACGGGTCGCATGGC    1.097117
AAAGATGAGTCATGCT    1.968045
AAAGATGAGTCTTGCA    2.006473
Name: Ppp1r1a, dtype: float64

In [56]:
model = Ridge(alpha=alpha, random_state=123)
model.fit(Data, Label)

Ridge(alpha=10, random_state=123)

In [57]:
model.coef_

array([ 0.15996642, -0.30617319, -0.84029342, -0.79090283, -0.97645844,
        0.00812149,  0.24807391,  0.45374527,  0.41518154,  0.2729266 ,
        0.4752087 ,  0.23000366, -1.87602611])

In [58]:
model.intercept_

4.352930773784504

In [59]:
model.score(Data,Label)

0.7133552781969135

Why regression regults doesn't make sense???

In [129]:
#input are the inputed counts of cells in beta
simulation_input = gem_sim_input[cells_in_beta] 

In [130]:
gem_ = simulation_input.copy()

In [131]:
#do simulation
delta_input = simulation_input - gem_

In [132]:
delta_simulated = delta_input.copy()

In [133]:
coef_matrix = oracle.coef_matrix_per_cluster['Beta']

In [134]:
n_propagation = 2

In [135]:
step={}
delta_sim={}
for i in range(n_propagation):
    if np.all((delta_simulated == 0)):
        step[i] = gem.dot(coef_matrix)
    else:
        delta_simulated = delta_simulated.dot(coef_matrix)
        delta_simulated[delta_input != 0] = delta_input
        # gene expression cannot be negative. adjust delta values to make sure that gene expression are not negative values.
        step[i] = gem + delta_simulated
    #step[i] = step[i].copy()
    step[i][step[i]<0] = 0
    delta_simulated = step[i] - gem
    delta_sim[i]=delta_simulated

In [165]:
gem_imputed[gem_imputed['Peg3']<1]['Peg3']

AAAGATGCAGCGATCC    0.982184
AACACGTCACGAAATA    0.990860
AACTGGTGTGCCTGTG    0.992547
AGCAGCCGTCAACATC    0.989449
AGCTTGAAGGTGATAT    0.939502
AGCTTGATCCCACTTG    0.992104
ATTCTACCATCCGCGA    0.993864
CACCACTGTCGGCTCA    0.986095
CCATGTCCACCAGTTA    0.971043
CCGTGGATCCAAGCCG    0.927571
CGAACATTCGGCGGTT    0.971629
CGCGGTAAGCCCAGCT    0.998932
CGTCAGGGTAGGCATG    0.998625
CTACACCGTCGCCATG    0.968538
CTCATTACAAGCGCTC    0.972493
CTGCCTAGTCTCCATC    0.936110
CTGCTGTCAGATGGGT    0.985005
GACCTGGAGTACGATA    0.976714
GACGGCTAGTTTAGGA    0.975778
GCATGATCATTGAGCT    0.970974
GCTGCGAAGTATTGGA    0.987594
GGAATAAGTTCGGGCT    0.944260
GGACATTCAAGTTCTG    0.999298
GGCAATTAGACTAGGC    0.995277
GGCCGATAGGGCACTA    0.968686
GTCAAGTCACACATGT    0.940388
GTGCATAAGCTCCTCT    0.958437
TACACGAAGAGCCCAA    0.971276
TCGCGTTAGATCTGAA    0.969795
TGGCGCATCACAACGT    0.966291
TGTGTTTGTAGCCTCG    0.967527
TTCCCAGAGGATTCGG    0.988006
Name: Peg3, dtype: float64

In [137]:
delta_sim[0][reggenes]

,Tfcp2,Zfp90,Nr4a2,Zfp467,Vdr,Nr1h4,Relb,Ovol2,Ddit3,E2f1,Hnf4a,Hivep1,Peg3
AAACGGGCAAAGAATC,0.0,0.0,0.0,0.0,-0.129002,0.0,0.0,0.0,-0.100962,0.0,0.0,0.0,-1.119581
AAACGGGGTACAGTTC,0.0,0.0,0.0,0.0,-0.113693,0.0,0.0,0.0,-0.087693,0.0,0.0,0.0,-1.161246
AAACGGGTCGCATGGC,0.0,0.0,0.0,0.0,-0.114713,0.0,0.0,0.0,-0.156562,0.0,0.0,0.0,-1.067991
AAAGATGAGTCATGCT,0.0,0.0,0.0,0.0,-0.129677,0.0,0.0,0.0,-0.139800,0.0,0.0,0.0,-1.062420
AAAGATGAGTCTTGCA,0.0,0.0,0.0,0.0,-0.102425,0.0,0.0,0.0,-0.180799,0.0,0.0,0.0,-1.116408
...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTATGCTCTCTGCTG,0.0,0.0,0.0,0.0,-0.083601,0.0,0.0,0.0,-0.167058,0.0,0.0,0.0,-1.234090
TTTCCTCAGTGTCCCG,0.0,0.0,0.0,0.0,-0.137502,0.0,0.0,0.0,-0.178663,0.0,0.0,0.0,-1.124203
TTTGCGCTCGCCTGTT,0.0,0.0,0.0,0.0,-0.151260,0.0,0.0,0.0,-0.094754,0.0,0.0,0.0,-1.113583
TTTGGTTCAAATTGCC,0.0,0.0,0.0,0.0,-0.139219,0.0,0.0,0.0,-0.165457,0.0,0.0,0.0,-1.161719


In [147]:
step[0]['Peg3']

AAACGGGCAAAGAATC    0.468470
AAACGGGGTACAGTTC    0.257514
AAACGGGTCGCATGGC    0.354862
AAAGATGAGTCATGCT    0.264096
AAAGATGAGTCTTGCA    0.258339
                      ...   
TTTATGCTCTCTGCTG    0.290821
TTTCCTCAGTGTCCCG    0.227488
TTTGCGCTCGCCTGTT    0.419504
TTTGGTTCAAATTGCC    0.214406
TTTGGTTTCCTTTCGG    0.405922
Name: Peg3, Length: 591, dtype: float64

In [145]:
np.asarray(delta_sim[0]['Peg3']>0) #delta_sim[0]['Peg3']

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [121]:
gem.dot(coef_matrix)['Ppp1r1a']

AAACGGGCAAAGAATC   -3.254433
AAACGGGGTACAGTTC   -2.675717
AAACGGGTCGCATGGC   -2.821439
AAAGATGAGTCATGCT   -2.518572
AAAGATGAGTCTTGCA   -2.542743
                      ...   
TTTATGCTCTCTGCTG   -2.910649
TTTCCTCAGTGTCCCG   -2.523709
TTTGCGCTCGCCTGTT   -3.192387
TTTGGTTCAAATTGCC   -2.550890
TTTGGTTTCCTTTCGG   -3.387702
Name: Ppp1r1a, Length: 591, dtype: float64

In [122]:
step[0]['Ppp1r1a']

AAACGGGCAAAGAATC    0.0
AAACGGGGTACAGTTC    0.0
AAACGGGTCGCATGGC    0.0
AAAGATGAGTCATGCT    0.0
AAAGATGAGTCTTGCA    0.0
                   ... 
TTTATGCTCTCTGCTG    0.0
TTTCCTCAGTGTCCCG    0.0
TTTGCGCTCGCCTGTT    0.0
TTTGGTTCAAATTGCC    0.0
TTTGGTTTCCTTTCGG    0.0
Name: Ppp1r1a, Length: 591, dtype: float64

In [123]:
step[1]['Ppp1r1a']

AAACGGGCAAAGAATC    2.707362
AAACGGGGTACAGTTC    4.341562
AAACGGGTCGCATGGC    3.147707
AAAGATGAGTCATGCT    4.029754
AAAGATGAGTCTTGCA    4.125833
                      ...   
TTTATGCTCTCTGCTG    4.037486
TTTCCTCAGTGTCCCG    4.516189
TTTGCGCTCGCCTGTT    2.941872
TTTGGTTCAAATTGCC    4.639041
TTTGGTTTCCTTTCGG    3.121559
Name: Ppp1r1a, Length: 591, dtype: float64

In [124]:
Label.head()

AAACGGGCAAAGAATC    0.522952
AAACGGGGTACAGTTC    2.088425
AAACGGGTCGCATGGC    1.097117
AAAGATGAGTCATGCT    1.968045
AAAGATGAGTCTTGCA    2.006473
Name: Ppp1r1a, dtype: float64

In [127]:
gem.dot(coef_matrix)[reggenes]

,Tfcp2,Zfp90,Nr4a2,Zfp467,Vdr,Nr1h4,Relb,Ovol2,Ddit3,E2f1,Hnf4a,Hivep1,Peg3
AAACGGGCAAAGAATC,0.062066,0.092348,0.119933,0.127558,0.089112,0.008502,0.013756,0.122148,-0.000057,0.038661,0.067924,0.061421,0.468470
AAACGGGGTACAGTTC,0.080351,0.059268,0.063167,0.042520,-0.001214,0.017316,0.019812,0.119374,-0.000013,0.063359,0.117505,0.087700,0.257514
AAACGGGTCGCATGGC,0.030334,0.076545,0.107376,0.120368,0.031336,0.031592,0.000000,0.161057,-0.000011,0.028561,0.069866,0.065906,0.354862
AAAGATGAGTCATGCT,0.058535,0.050540,0.051827,0.051987,0.001431,0.019153,0.020692,0.149305,-0.000022,0.031537,0.060409,0.088753,0.264096
AAAGATGAGTCTTGCA,0.100374,0.050300,0.059903,0.057831,0.001003,0.011138,0.007248,0.169970,-0.000013,0.065875,0.089835,0.078189,0.258339
...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTATGCTCTCTGCTG,0.096747,0.075234,0.110646,0.083808,0.018188,0.022668,0.007990,0.145625,-0.000021,0.029498,0.105715,0.088889,0.290821
TTTCCTCAGTGTCCCG,0.064876,0.047284,0.042636,0.049596,-0.001698,0.029203,0.036431,0.149012,-0.000022,0.044281,0.106563,0.050972,0.227488
TTTGCGCTCGCCTGTT,0.029492,0.051041,0.146769,0.137108,0.067041,0.037078,0.007206,0.093304,0.000000,0.014622,0.093325,0.032431,0.419504
TTTGGTTCAAATTGCC,0.058780,0.047947,0.050103,0.027656,0.000372,0.031020,0.031885,0.157955,-0.000009,0.069396,0.121228,0.048854,0.214406


In [128]:
step[0][reggenes]

,Tfcp2,Zfp90,Nr4a2,Zfp467,Vdr,Nr1h4,Relb,Ovol2,Ddit3,E2f1,Hnf4a,Hivep1,Peg3
AAACGGGCAAAGAATC,0.062066,0.092348,0.119933,0.127558,0.089112,0.008502,0.013756,0.122148,0.0,0.038661,0.067924,0.061421,0.468470
AAACGGGGTACAGTTC,0.080351,0.059268,0.063167,0.042520,0.000000,0.017316,0.019812,0.119374,0.0,0.063359,0.117505,0.087700,0.257514
AAACGGGTCGCATGGC,0.030334,0.076545,0.107376,0.120368,0.031336,0.031592,0.000000,0.161057,0.0,0.028561,0.069866,0.065906,0.354862
AAAGATGAGTCATGCT,0.058535,0.050540,0.051827,0.051987,0.001431,0.019153,0.020692,0.149305,0.0,0.031537,0.060409,0.088753,0.264096
AAAGATGAGTCTTGCA,0.100374,0.050300,0.059903,0.057831,0.001003,0.011138,0.007248,0.169970,0.0,0.065875,0.089835,0.078189,0.258339
...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTATGCTCTCTGCTG,0.096747,0.075234,0.110646,0.083808,0.018188,0.022668,0.007990,0.145625,0.0,0.029498,0.105715,0.088889,0.290821
TTTCCTCAGTGTCCCG,0.064876,0.047284,0.042636,0.049596,0.000000,0.029203,0.036431,0.149012,0.0,0.044281,0.106563,0.050972,0.227488
TTTGCGCTCGCCTGTT,0.029492,0.051041,0.146769,0.137108,0.067041,0.037078,0.007206,0.093304,0.0,0.014622,0.093325,0.032431,0.419504
TTTGGTTCAAATTGCC,0.058780,0.047947,0.050103,0.027656,0.000372,0.031020,0.031885,0.157955,0.0,0.069396,0.121228,0.048854,0.214406


In [ ]:
gem_imputed = _adata_to_df(self.adata, "imputed_count")

        self.adata.layers["simulation_input"] = self.adata.layers["imputed_count"].copy()
        self.alpha_for_trajectory_GRN = alpha
        self.GRN_unit = GRN_unit

        if use_cluster_specific_TFdict & (self.cluster_specific_TFdict is not None):
            self.coef_matrix_per_cluster = {}
            cluster_info = self.adata.obs[self.cluster_column_name]

            print(f"fitting GRN again...")
            for cluster in np.unique(cluster_info):
                print(f"calculating GRN in {cluster}")
                cells_in_the_cluster_bool = (cluster_info == cluster)
                gem_ = gem_imputed[cells_in_the_cluster_bool]
                self.coef_matrix_per_cluster[cluster] = _getCoefMatrix(gem=gem_,
                                                                       TFdict=self.cluster_specific_TFdict[cluster],
                                                                       alpha=alpha)